In [1]:
import requests
from bs4 import BeautifulSoup
import math
import pandas as pd
import re
import gspread
from oauth2client.service_account import ServiceAccountCredentials

In [2]:
l=[]
o={}
k=[]
headers={"User-Agent":"Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/107.0.0.0 Safari/537.36"}
target_url='https://www.linkedin.com/jobs/search?keywords=Python&location=India&geoId=102713980&trk=public_jobs_jobs-search-bar_search-submit&position=1&pageNum={}'
# the target link can be replaced with any desired job filter
runs = 10
for i in range(runs):
    res = requests.get(target_url.format(i))
    if res.status_code == 200:
        soup = BeautifulSoup(res.text, 'html.parser')
        job_data = soup.find("code", id="currentJobId")
        if job_data:
            content = job_data.string
            if content:
                job_id = content.strip()
                l.append(job_id)
l = list(set(l))
print(l)

['3731835443', '3748931980']


In [3]:
# from lxml import html
# def extract_text_from_xpath(url, xpaths):
#     response = requests.get(url)
#     if response.status_code == 200:
#         tree = html.fromstring(response.content.decode('utf-8'))
#         extracted_text = []
#         for xpath in xpaths:
#             elements = tree.xpath(xpath)
#             if elements:
#                 extracted_text.extend([element.text_content() for element in elements])
#         return extracted_text
#     else:
#         print(f"Failed to fetch the URL. Status code: {response.status_code}")
#         return []

In [4]:
#target_url='https://www.linkedin.com/jobs-guest/jobs/api/seeMoreJobPostings/{}'
#for j in range(0,len(l)):
#    company_xpaths = [f'/html/body/li[{i}]/div/div[2]/h4/a' for i in range(26)]
#    o['company'] = extract_text_from_xpath(target_url.format(l[j]), company_xpaths)
#    location_xpaths = [f'/html/body/li[{i}]/div/div[2]/div/span' for i in range(26)]
#    o['job-location'] = extract_text_from_xpath(target_url.format(l[j]), location_xpaths)
#    title_xpaths = [f'/html/body/li[{i}]/div/div[2]/h3' for i in range(26)]
#    o['job-title'] = extract_text_from_xpath(target_url.format(l[j]), title_xpaths)
#    url_xpaths = [f'/html/body/li[{i}]/div/a' for i in range(26)]
#    o['job-url'] = extract_text_from_xpath(target_url.format(l[j]), url_xpaths)
#    k.append(o)

#df = pd.DataFrame(k)
#df.to_csv('linkedinjobs.csv', index=False, encoding='utf-8')
#print(k)

xpath method didn't word

In [5]:
target_url='https://www.linkedin.com/jobs-guest/jobs/api/jobPosting/{}'

# to prevent itself from scraping, linkedin keeps on updating its HTML code. So, the code below might not work in future and has to be updated

pattern = r'for the(.*?)role at'
    
for j in range(0,len(l)):

    resp = requests.get(target_url.format(l[j]))
    soup=BeautifulSoup(resp.text,'html.parser')

    try:
        company = soup.find("p", class_="sign-up-modal__sub-header").text.strip()
        o["company"]=company.split('role at', 1)[1].strip()
    except:
        o["company"]=None

    try:
        title = soup.find("p", class_="sign-up-modal__sub-header").text.strip()
        o["job-title"]=re.search(pattern, title).group(1).strip()
    except:
        o["job-title"]=None

    try:
        o["job-location"]=soup.find("span", class_="topcard__flavor topcard__flavor--bullet").text.strip()
    except:
        o["job-location"]=None
        
    try:
        o["job-url"]=soup.a.get('href')
    except:
        o["job-url"]=None

    k.append(o)
    o={}

print(k)

[{'company': 'Infosys', 'job-title': 'Python Lead', 'job-location': 'Bengaluru East, Karnataka, India', 'job-url': 'https://in.linkedin.com/company/infosys?trk=public_jobs_topcard_logo'}, {'company': 'Sprinklr', 'job-title': 'Product Analyst', 'job-location': 'Gurgaon, Haryana, India', 'job-url': 'https://www.linkedin.com/company/sprinklr?trk=public_jobs_topcard_logo'}]


In [6]:
scope = ["https://www.googleapis.com/auth/spreadsheets", "https://www.googleapis.com/auth/drive"]

credentials = ServiceAccountCredentials.from_json_keyfile_name(r'C:\Users\rohit\Python\skuad_assignment\linkedin-fetcher-288b8b77c6bc.json', scope)

gc = gspread.authorize(credentials)

sheet = gc.open("linkedin_fetcher")

worksheet = sheet.get_worksheet(0)

existing_data = worksheet.get_all_values()

for item in k:
    values = [item['company'], item['job-title'], item['job-location'], item['job-url']]
    existing_data.append(values)

worksheet.update("A1", existing_data)

C:\Users\rohit\AppData\Local\Temp\ipykernel_22344\3298829081.py:17: DeprecationWarning: [Deprecated][in version 6.0.0]: Method signature's arguments 'range_name' and 'values' will change their order. We recommend using named arguments for minimal impact. In addition, the argument 'values' will be mandatory of type: 'List[List]'. (ex) Worksheet.update(values = [[]], range_name=) 
  worksheet.update("A1", existing_data)


{'spreadsheetId': '1Sxe3mhzOeH43QIRL2J5ozth3kS2z0gb-LMXXnX4T_Yw',
 'updatedRange': 'Sheet1!A1:D2',
 'updatedRows': 2,
 'updatedColumns': 4,
 'updatedCells': 8}